Hypothesis testing is a statistical method used to make inferences or decisions about a population based on sample data.
'H0' which is the null hypothesis and it is a statement defined for the sake of our arguement or it is defined as the default stance.
'H1' is the alternate hypothesis and it is something which should be proved or disproved.

Let's not dive into much theory instead focus on learning with practice.

## When are we supposed to use this techniques or testing?
Yes, it is opted for situations demanding making inferences on a population based on a given set of sample data.
Here are the steps involved for the same:
* Gather the necessary data required for the hypothesis test.
* Define Null (H0) and Alternative Hypothesis (H1 or Ha).
* Choose the Significance Level (α), which is the probability of rejecting the null hypothesis when it is true.
* Select the appropriate  statistical tests. 
  Examples include t-tests for comparing means, chi-square tests for categorical data, and ANOVA for comparing means across more   than two groups.
* Perform the chosen  statistical test on your data.
* Determine the p-value and interpret the results of your statistical tests.

Now let's start the procedure.

In [2]:
import pandas as pd
import numpy as np

In [3]:
from scipy.stats import ttest_ind

In [4]:
df = pd.read_csv('website_ab_test.csv')
df.head(3)

,Theme,Click Through Rate,Conversion Rate,Bounce Rate,Scroll_Depth,Age,Location,Session_Duration,Purchases,Added_to_Cart
0,Light Theme,0.054920,0.282367,0.405085,72.489458,25,Chennai,1535,No,Yes
1,Light Theme,0.113932,0.032973,0.732759,61.858568,19,Pune,303,No,Yes
2,Dark Theme,0.323352,0.178763,0.296543,45.737376,47,Chennai,563,Yes,Yes


### About the dataset
An online bookstore is looking to optimize its website design to improve user engagement and ultimately increase book purchases. The website currently offers two themes for its users: “Light Theme” and “Dark Theme.” The bookstore’s data science team wants to conduct an A/B testing experiment to determine which theme leads to better user engagement and higher conversion rates for book purchases. (A/B testing, also known as split testing, is a method of comparing two versions of a webpage or other user experience to determine which one performs better).

The data collected by the bookstore contains user interactions and engagement metrics for both the Light Theme and Dark Theme. The dataset includes the following key features:

- Theme: dark or light
- Click Through Rate: The proportion of the users who click on links or buttons on the website.
- Conversion Rate: The percentage of users who signed up on the platform after visiting for the first time.
- Bounce Rate: The percentage of users who leave the website without further interaction after visiting a single page.
- Scroll Depth: The depth to which users scroll through the website pages.
- Age: The age of the user.
- Location: The location of the user.
- Session Duration: The duration of the user’s session on the website.
- Purchases: Whether the user purchased the book (Yes/No).
- Added_to_Cart: Whether the user added books to the cart (Yes/No).

Our task is to identify which theme, Light Theme or Dark Theme, yields better user engagement, purchases and conversion rates. 

We need to determine if there is a statistically significant difference in the key metrics between the two themes.

In [5]:
# Let's find out the summary
Summary = {
    'Total no. of data points': df.shape[0],
    'Total no. of columns':df.shape[1],
    'Missing values': df.isna().sum(),
    'Analysis of numerical datas':df.describe()
}

Summary

{'Total no. of data points': 1000,
 'Total no. of columns': 10,
 'Missing values': Theme                 0
 Click Through Rate    0
 Conversion Rate       0
 Bounce Rate           0
 Scroll_Depth          0
 Age                   0
 Location              0
 Session_Duration      0
 Purchases             0
 Added_to_Cart         0
 dtype: int64,
 'Analysis of numerical datas':        Click Through Rate  Conversion Rate  Bounce Rate  Scroll_Depth  \
 count         1000.000000      1000.000000  1000.000000   1000.000000   
 mean             0.256048         0.253312     0.505758     50.319494   
 std              0.139265         0.139092     0.172195     16.895269   
 min              0.010767         0.010881     0.200720     20.011738   
 25%              0.140794         0.131564     0.353609     35.655167   
 50%              0.253715         0.252823     0.514049     51.130712   
 75%              0.370674         0.373040     0.648557     64.666258   
 max              0.499989    

Now we got the basic maths behind the data after carefully reviewing the above steps. Our primary concerns is which theme should be chosen? First we need to identify the relevant features and then review it by performing hypothesis testing.

In [6]:
# Select only numeric columns for the mean calculation
numeric_cols = df.select_dtypes(include='number').columns
df_numeric = df[numeric_cols]

# Include the 'Theme' column for grouping
df_numeric['Theme'] = df['Theme']

# Group by 'Theme' and calculate the mean for numeric columns
theme_performance = df_numeric.groupby('Theme').mean()

# Sort the data based on 'Conversion Rate' for better comparison
theme_performance_sorted = theme_performance.sort_values(by='Conversion Rate', ascending=False)

print(theme_performance_sorted)

             Click Through Rate  Conversion Rate  Bounce Rate  Scroll_Depth  \
Theme                                                                         
Light Theme            0.247109         0.255459     0.499035     50.735232   
Dark Theme             0.264501         0.251282     0.512115     49.926404   

                   Age  Session_Duration  
Theme                                     
Light Theme  41.734568        930.833333  
Dark Theme   41.332685        919.482490  


C:\Users\devik\AppData\Local\Temp\ipykernel_3976\2936753225.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_numeric['Theme'] = df['Theme']


From these insights, it appears that the Light Theme slightly outperforms the Dark Theme in terms of Conversion Rate, Bounce Rate, Scroll Depth, and Session Duration, while the Dark Theme leads in Click Through Rate. However, the differences are relatively minor across all metrics.

## Let's start hypothesis testing

We’ll use a significance level (alpha) of 0.05 for our hypothesis testing. It means we’ll consider a result statistically significant if the p-value from our test is less than 0.05.

Let's start with the conversion rate betweent the two themes:
- Null Hypothesis (H0): There is no difference in Conversion Rates between the Light Theme and Dark Theme.
- Alternative Hypothesis (Ha): There is a difference in Conversion Rates between the Light Theme and Dark Theme.

In [7]:
# extracting conversion rates for both themes
conversion_rates_light = df[df['Theme'] == 'Light Theme']['Conversion Rate']
conversion_rates_dark = df[df['Theme'] == 'Dark Theme']['Conversion Rate']

# performing a two-sample t-test
t_stat, p_value = ttest_ind(conversion_rates_light, conversion_rates_dark, equal_var=False)

t_stat, p_value

(0.4748494462782632, 0.6349982678451778)

The result of the two-sample t-test gives a p-value of approximately 0.635. Since this p-value is much greater than our significance level of 0.05, we do not have enough evidence to reject the null hypothesis. Therefore, we conclude that there is no statistically significant difference in Conversion Rates between the Light Theme and Dark Theme based on the data provided.

Now, let’s conduct hypothesis testing based on the Click Through Rate (CTR) to see if there’s a statistically significant difference between the Light Theme and Dark Theme regarding how often users click through. Our hypotheses remain structured similarly:

- Null Hypothesis (H0): There is no difference in Click Through Rates between the Light Theme and Dark Theme.
- Alternative Hypothesis (Ha): There is a difference in Click Rates between the Light Theme and Dark Theme.

In [8]:
# extracting click through rates for both themes
ctr_light = df[df['Theme'] == 'Light Theme']['Click Through Rate']
ctr_dark = df[df['Theme'] == 'Dark Theme']['Click Through Rate']

# performing a two-sample t-test
t_stat_ctr, p_value_ctr = ttest_ind(ctr_light, ctr_dark, equal_var=False)

t_stat_ctr, p_value_ctr

(-1.9781708664172253, 0.04818435371010704)

The two-sample t-test for the Click Through Rate (CTR) between the Light Theme and Dark Theme yields a p-value of approximately 0.048. This p-value is slightly below our significance level of 0.05, indicating that there is a statistically significant difference in Click Through Rates between the Light Theme and Dark Theme, with the Dark Theme likely having a higher CTR given the direction of the test statistic.


Now, let’s perform Hypothesis Testing based on two other metrics: bounce rate and scroll depth, which are important metrics for analyzing the performance of a theme or a design on a website. I’ll first perform these statistical tests and then create a table to show the report of all the tests we have done:

In [9]:
# extracting bounce rates for both themes
bounce_rates_light = df[df['Theme'] == 'Light Theme']['Bounce Rate']
bounce_rates_dark = df[df['Theme'] == 'Dark Theme']['Bounce Rate']

# performing a two-sample t-test for bounce rate
t_stat_bounce, p_value_bounce = ttest_ind(bounce_rates_light, bounce_rates_dark, equal_var=False)

# extracting scroll depths for both themes
scroll_depth_light = df[df['Theme'] == 'Light Theme']['Scroll_Depth']
scroll_depth_dark = df[df['Theme'] == 'Dark Theme']['Scroll_Depth']

# performing a two-sample t-test for scroll depth
t_stat_scroll, p_value_scroll = ttest_ind(scroll_depth_light, scroll_depth_dark, equal_var=False)

# creating a table for comparison
comparison_table = pd.DataFrame({
    'Metric': ['Click Through Rate', 'Conversion Rate', 'Bounce Rate', 'Scroll Depth'],
    'T-Statistic': [t_stat_ctr, t_stat, t_stat_bounce, t_stat_scroll],
    'P-Value': [p_value_ctr, p_value, p_value_bounce, p_value_scroll]
})

comparison_table

,Metric,T-Statistic,P-Value
0,Click Through Rate,-1.978171,0.048184
1,Conversion Rate,0.474849,0.634998
2,Bounce Rate,-1.201888,0.229692
3,Scroll Depth,0.756228,0.449692


- Click Through Rate: The test reveals a statistically significant difference, with the Dark Theme likely performing better (P-Value = 0.048).
- Conversion Rate: No statistically significant difference was found (P-Value = 0.635).
- Bounce Rate: There’s no statistically significant difference in Bounce Rates between the themes (P-Value = 0.230).
- Scroll Depth: Similarly, no statistically significant difference is observed in Scroll Depths (P-Value = 0.450).

In summary, while the two themes perform similarly across most metrics, the Dark Theme has a slight edge in terms of engaging users to click through. For other key performance indicators like Conversion Rate, Bounce Rate, and Scroll Depth, the choice between a Light Theme and a Dark Theme does not significantly affect user behaviour according to the data provided.